<a href="https://colab.research.google.com/github/RodolfoFerro/ai-python/blob/master/AI_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Taller de AI con Python

Taller de AI con Python compartido para la comunidad de AI Gaming LATAM.

Código documentado por **Rodolfo Ferro**, Google Developer Expert en Machine Learning.

#### Redes:
- GitHub: [RodolfoFerro](https://github.com/RodolfoFerro)
- Twitter: [FerroRodolfo](https://twitter.com/FerroRodolfo)
- Instagram: [rodo_ferro](https://www.instagram.com/rodo_ferro/)

## Descarga de datos

Los datos se encuentran disponibles de manera pública en https://nasadata.blob.core.windows.net/nasarocks/Data.zip. El siguiente comando descargará directamente el conjunto de imágenes y descomprimirá la carpeta en Google Colab.

In [ ]:
!wget https://nasadata.blob.core.windows.net/nasarocks/Data.zip
!unzip Data

## Código

El código a continuación va a realizar toda la magia de entrenar una red neuronal para clasificar las imágenes que hemos descargado.

> **Nota:** Google Colab ya cuenta con un montón de paquetería instalada, así que con ello podemos omitir cualquier proceso de instalación y brincar directo a trabajar con el código.

Comenzaremos importando las paqueterías que vamos a utilizar:

In [ ]:
# Matplotlib será nuestra biblioteca para crear gráficas.
# Ésta contiene un módulo para ploteo que importeremos con el alias de plt.
# Además, configuraremos parámetros sobre la calidad de las figuras.
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.dpi'] = 300

# Ahora importamos numpy, el paquete numérico (seguramente el más 
# importante para ello) de Python.
import numpy as np

# Continuamos importando PyTorch, una biblioteca desarrololada por Facebook 
# para el desarrollo de modelos de IA con redes neuronales.
import torch
# Para instanciar una red neuronal y agregar un optimizador.
from torch import nn, optim

# Para diferenciación en el proceso de cómputo de gradientes.
from torch.autograd import Variable
import torch.nn.functional as F

# Agregamos funciones para sampleo de muestras (para las imágenes).
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader

# Torchvision incluye procesamientos para trabajar con imágenes.
import torchvision
from torchvision import datasets, transforms, models

# Finalmente, importamos la Python Image Library para poder cargar las imágenes.
from PIL import Image 

Especificamos el folder de trabajo:

In [ ]:
data_dir = './Data'

Creamos una función que cargue las imágenes y las divida en un conjunto para entrenar la red neuronal y otro para probar los resultados. 

> **Nota:** Normalmente partimos los datasets en dos subconjuntos, uno para entrenar el modelo con datos que sí le mostramos, y otro para medir las predicciones de la red con datos que nunca ha visto. Con esto, intentamos evitar sesgos a la hora de medir la precisión en las predicciones al tener una métrica un poco más real.

In [ ]:
def load_split_train_test(datadir, valid_size=.2):
    
    # Cargamos funciones que procesen las imágenes antes de alimentar 
    # a la red neuronal, pues necesitamos definir un tamaño estándar
    # y convertir las imagen a un tensor (generalizaciónd e una matriz).
    train_transforms = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.Resize(224),
        transforms.ToTensor(),
    ])

    test_transforms = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.Resize(224),
        transforms.ToTensor(),
    ])

    # Especificamos de dónde carga las imágenes y qué transformaciones aplica.
    train_data = datasets.ImageFolder(datadir, transform=train_transforms)
    test_data = datasets.ImageFolder(datadir, transform=test_transforms)

    # Definimos algunos parámetros para partir el conjunto de datos y
    # aleatorizamos el orden de las imágenes.
    num_train = len(train_data)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))
    np.random.shuffle(indices)

    # Procedemos a separar los datos en conjuntos de entrenamiento y
    # prueba utilizando utilería de pytorch.
    train_idx, test_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(test_idx)
    
    trainloader = DataLoader(train_data, sampler=train_sampler, batch_size=16)
    testloader = DataLoader(test_data, sampler=test_sampler, batch_size=16)
    
    return trainloader, testloader

Utilizamos la función que acabamos de crear para partir las imágenes e conjuntos de entrenamiento y pruebas:

In [ ]:
trainloader, testloader = load_split_train_test(data_dir, .2)
print(trainloader.dataset.classes)

Vemos que en general hay sólo 2 clases de rocas por clasificar.

Creamos otro proceso de transformaciones para las imágenes, pues nos servirá para extraer algunas imágenes de muestra.

In [ ]:
test_transforms = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.Resize(224),
        transforms.ToTensor(),
    ])

Y creamos una función más que nos ayude a extraer dicha muestra de imágenes:

In [ ]:
def get_random_images(num):
    data = datasets.ImageFolder(data_dir, transform=test_transforms)
    classes = data.classes
    indices = list(range(len(data)))
    np.random.shuffle(indices)
    idx = indices[:num]
    
    sampler = SubsetRandomSampler(idx)
    loader = torch.utils.data.DataLoader(data, sampler=sampler, batch_size=num)
    dataiter = iter(loader)
    images, labels = dataiter.next()
    
    return images, labels

Utilizamos esta función y visualizamos una muestra para ver cómo son las imágenes que estaremos utilizando con la red:

In [ ]:
images, labels = get_random_images(5)  # Muestreo de imágenes.
to_pil = transforms.ToPILImage()       # Transformación.
fig = plt.figure(figsize=(20, 20))     # Creamos una figura vacía.
classes = trainloader.dataset.classes  # Cargamos las clases.

# Procedemos a añadir imágenes en una cadrícula dentro de la figura.
for k in range(len(images)):
    image = to_pil(images[k])
    sub = fig.add_subplot(1, len(images), k + 1)
    plt.axis('off')
    plt.imshow(image)
plt.show()

Claramente el procesamiento para la selección de buenas regiones en las imágenes puede ser mejorado...

Continuamos a configurar algunas cosas de pytorch, como el que use GPU o CPU dependiendo de tu máquina. 

Google Colab puede utilizar ambos, si estás ejecutando este cuaderno, previamente lo he configurado para que utilice GPUs.

Además, utilizaremos un modelo precargado y pre-entrenado llamado ResNet (puedes investigar más sobre el mismo y su arquitectura, es una red neuronal convolucional, así que funciona bien con imágenes).

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


model = models.resnet50(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

Con el modelo ya cargado, procedemos a agregar una capa "full connected", especificando algunos parámtros, como el número de neuronas, función de activación, etc.

In [ ]:
model.fc = nn.Sequential(
    nn.Linear(2048, 512),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(512, 2),
    nn.LogSoftmax(dim=1)
)

# Especificamos función de pérdida y optimizador.
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.003)
model.to(device)
print('Done.')

Y finalmente procedemos a entrenar la red neuronal con nuestros propios datos.

En general, hasta este punto ya me dio flojera documentar paso a paso lo que sigue para el entrenamiento (se me enfrían los tacos), pero en escencia lo que sucede en estos pasos es que estima una predicción inicial con los pesos precargados y calcula un error, utiliza ese error en el optimizador (gracias al gradiente) y varía los pesos hacia donde apunta el vector gradiente para intentar minimizar el error.

> Si tienes dudas más específicas sobre qué hace cada línea, puedes escribirme.

In [ ]:
# Definimos algunos hyperparámetros.
epochs = 5
steps = 0
running_loss = 0
print_every = 5
train_losses, test_losses = [], []

# Procedemos a entrenar el modelo.
for epoch in range(epochs):
    for inputs, labels in trainloader:

        steps += 1
        print('Training step ', steps)
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in testloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    test_loss += batch_loss.item()
                    
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

            train_losses.append(running_loss/len(trainloader))
            test_losses.append(test_loss/len(testloader))                    
            print(f"Epoch {epoch+1}/{epochs}.. "
                    f"Train loss: {running_loss/print_every:.3f}.. "
                    f"Test loss: {test_loss/len(testloader):.3f}.. "
                    f"Test accuracy: {accuracy/len(testloader):.3f}")
            running_loss = 0
            model.train()

Ya que hemos entrenado el modelo, podemos medir la precisión de entrenamiento:

In [ ]:
print(accuracy / len(testloader))

Si deseas guardar el modelo entrenado y descargarlo para correrlo desde una aplicación web o algo así, podemos hacerlo como sigue:

In [ ]:
torch.save(model, 'aerialmodel.pth')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.load('aerialmodel.pth')

Ahora creamos una función para invocar al modelo y realizar la magia de inferencia para predecir las clases de imágenes de muestra.

Dicha función sólo devolverá la clase predicha.

In [ ]:
def predict_image(image):
    image_tensor = test_transforms(image).float()
    image_tensor = image_tensor.unsqueeze_(0)
    input = Variable(image_tensor)
    input = input.to(device)
    output = model(input)
    index = output.data.cpu().numpy().argmax()
    
    return index

Cargamos algunas imágenes random de muestra y utilizamos el modelo que entrenamos para imprimir si la clase predicha es correcta:

In [ ]:
images, labels = get_random_images(5)  # Muestreo de imágenes.
to_pil = transforms.ToPILImage()       # Transformación.
classes = trainloader.dataset.classes  # Cargamos las clases.
fig = plt.figure(figsize=(20, 20))     # Creamos una figura vacía.

for k in range(len(images)):
    image = to_pil(images[k])
    index = predict_image(image)
    sub = fig.add_subplot(1, len(images), k + 1)
    res = int(labels[k]) == index
    sub.title.set_text(str(classes[index]) + ": " + str(res))
    plt.axis('off')
    plt.imshow(image)
plt.show()

Puedes ejecutar esta última celda varias veces para ver resultados de diferentes muestras de imágenes.